In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
# %env LANGCHAIN_TRACING_V2=true
# %env LANGCHAIN_API_KEY=...
# %env OPENAI_API_KEY=...

env: ANYWIDGET_HMR=1


In [2]:
import bs4
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Choose your LLM and Embedding Model

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOllama(model="llama3")

# embedding = OpenAIEmbeddings()
# embedding = HuggingFaceEmbeddings()
embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")

## Vector store

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of your corpus to create a retriever.
loader = WebBaseLoader(
    web_paths=(
        "https://anywidget.dev/en/getting-started/",
        "https://anywidget.dev/en/jupyter-widgets-the-good-parts/",
        "https://anywidget.dev/en/notebooks/counter/",
        "https://anywidget.dev/en/bundling/",
        "https://anywidget.dev/en/experimental/",
        "https://anywidget.dev/blog/introducing-anywidget/",
        "https://anywidget.dev/blog/anywidget-02/",
        "https://anywidget.dev/blog/a-year-with-anywidget/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("main-section")
        )
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

persist_directory = None  # To save the store to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=persist_directory)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# Load persistent store from disk
# vectorstore = Chroma(embedding_function=embedding, persist_directory="db")

## Chat history-aware vector store retriever chain

In [6]:
retriever_system_prompt = (
    "Given a chat history and the latest user question, "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

retriever_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# This chain takes a user question and the chat history, 
# asks the LLM to contextualize the question with the chat history
# then queries the vector store with the contextualized question
# to retrieve document snippets to use as context.
history_aware_retriever = create_history_aware_retriever(
    llm, vectorstore.as_retriever(), retriever_prompt
)

## Chat history-aware question-answer chain

In [7]:
qa_system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# This chain takes retrieved documents from a retriever and stuffs the 
# documents into the system prompt before taking in the chat history 
# and user question.
qa_chain = create_stuff_documents_chain(llm, qa_prompt)

## The retrieval-augmented generation (RAG) chain

In [8]:
# This chain retrieves query-relevant documents from the retriever chain 
# and passes them onto the q&a chain.
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

## Try it out!

In [9]:
import ipylangchat

In [10]:
widget = ipylangchat.ChatUIWidget(rag_chain)
widget

ChatUIWidget()